In [ ]:
# ===========================================
# 1. Spark config to read from ADLS (SAS)
# ===========================================
storage_account_name = "<YOUR_STORAGE_ACCOUNT>"
sas_token = "<your SAS token>"  
spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account_name}.dfs.core.windows.net", sas_token)

# ===========================================
# 2. Read Parquet From ADLS
# ===========================================
parquet_path = f"abfss://{your prediction data stored container name}@{storage_account_name}.dfs.core.windows.net/future_predictions.parquet?{sas_token}"
df_preds_spark = spark.read.parquet(parquet_path)

# ===========================================
# 3. (Optional) Transform/Subset DataFrame
# ===========================================
df_preds_spark = df_preds_spark.select("store_id", "product_id", "date", "predicted_sales_qty")

# ===========================================
# 4. Build JDBC Connection Properties
# ===========================================
jdbc_hostname = "YOUR_SQL_SERVER_NAME.database.windows.net"
jdbc_port = 1433
jdbc_database = "YOUR_DATABASE_NAME"

jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:{jdbc_port};databaseName={jdbc_database}"

db_properties = {
    "user": "YOUR_SQL_USERNAME",
    "password": "YOUR_SQL_PASSWORD",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

table_name = "dbo.FuturePredictions"

# ===========================================
# 5. Write Spark DataFrame to Azure SQL
# ===========================================
df_preds_spark.write \
  .mode("append") \
  .jdbc(url=jdbc_url, table=table_name, properties=db_properties)
